# Exercice 1.1.2 - Définition de Métriques

## Résumé et Conclusions

### Objectif :
Définir et implémenter **2 métriques différentes** sur le dataset Iris (150 échantillons × 4 features en cm).

### Métriques définies :

**1. Distance Euclidienne Normalisée** :
$$d_1(x, y) = \sqrt{\sum_{i=1}^{n} \frac{(x_i - y_i)^2}{\sigma_i^2}}$$
- Normalise chaque dimension par son écart-type
- Donne le même poids à toutes les features
- Valeurs typiques : 0 à ~10

**2. Distance de Manhattan Pondérée** :
$$d_2(x, y) = \sum_{i=1}^{n} w_i |x_i - y_i|$$
- Poids : $w = [0.1, 0.1, 0.4, 0.4]$ (priorité aux pétales)
- Robuste aux outliers (valeur absolue)
- Valeurs typiques : 0 à ~3

### Propriétés vérifiées :
| Propriété | $d_1$ | $d_2$ | Vérification |
|-----------|-------|-------|-------------|
| Positivité | ✓ | ✓ | Toutes distances ≥ 0 |
| Identité | ✓ | ✓ | d(x,x) = 0 |
| Symétrie | ✓ | ✓ | d(x,y) = d(y,x) |
| Inégalité triangulaire | ✓ | ✓ | d(x,z) ≤ d(x,y) + d(y,z) |

### Comportements différents :
- $d_1$ : Sensible aux différences quadratiques (pénalise les grands écarts)
- $d_2$ : Plus robuste, sensible aux pétales (pondération)
- Pour certaines paires, $d_1 > d_2$, pour d'autres $d_1 < d_2$
- Les deux métriques ordonnent différemment les paires de fleurs

### Conclusion :
Les 2 métriques sont valides mais capturent des aspects différents de la similarité. La distance euclidienne normalisée est plus sensible aux variations globales, tandis que la distance de Manhattan pondérée privilégie les caractéristiques des pétales.

---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## 1. Chargement et présentation du dataset

Nous utilisons le dataset **Iris** qui contient des mesures de fleurs :
- **sepal_length** : Longueur du sépale (cm)
- **sepal_width** : Largeur du sépale (cm)
- **petal_length** : Longueur du pétale (cm)
- **petal_width** : Largeur du pétale (cm)

In [ ]:
# Chargement du dataset
iris = load_iris()
df = pd.DataFrame(iris.data, columns=['sepal_length_cm', 'sepal_width_cm', 
                                       'petal_length_cm', 'petal_width_cm'])

print("Dataset Iris - Mesures de fleurs (toutes en cm)")
print("=" * 50)
print(df.head(10))
print(f"\nDimensions : {df.shape[0]} échantillons × {df.shape[1]} features")
print("\nStatistiques descriptives :")
print(df.describe().round(2))

## 2. Définition des deux métriques

### Métrique 1 : Distance Euclidienne Normalisée
On normalise d'abord les données (z-score) pour que chaque feature ait la même importance, puis on calcule la distance euclidienne.

$$d_1(x, y) = \sqrt{\sum_{i=1}^{n} \left(\frac{x_i - \mu_i}{\sigma_i} - \frac{y_i - \mu_i}{\sigma_i}\right)^2}$$

Cette métrique donne un **poids égal à toutes les features** après normalisation.

### Métrique 2 : Distance Manhattan Pondérée par la Variance
On pondère chaque feature par sa variance pour donner plus d'importance aux features avec plus de variabilité.

$$d_2(x, y) = \sum_{i=1}^{n} \sigma_i^2 \cdot |x_i - y_i|$$

Cette métrique donne **plus de poids aux features avec une grande variance** (notamment la longueur des pétales).

In [ ]:
# Calcul des statistiques pour les métriques
means = df.mean().values
stds = df.std().values
variances = df.var().values

print("Statistiques utilisées pour les métriques :")
print(f"Moyennes (cm) : {means.round(2)}")
print(f"Écarts-types (cm) : {stds.round(2)}")
print(f"Variances (cm²) : {variances.round(2)}")

In [ ]:
def metric1_euclidean_normalized(x, y, means, stds):
    """
    Métrique 1: Distance Euclidienne après normalisation z-score.
    Unité : sans dimension (les cm sont normalisés)
    """
    x_norm = (x - means) / stds
    y_norm = (y - means) / stds
    return np.sqrt(np.sum((x_norm - y_norm) ** 2))

def metric2_manhattan_weighted(x, y, variances):
    """
    Métrique 2: Distance Manhattan pondérée par la variance.
    Unité : cm³ (cm² de la variance × cm de la différence absolue)
    """
    return np.sum(variances * np.abs(x - y))

## 3. Calcul des distances pour toutes les paires

In [ ]:
n_samples = len(df)
data = df.values

# Stockage des distances
distances_m1 = []
distances_m2 = []
pairs = []

# Calcul de toutes les paires (i, j) avec i < j
for i in range(n_samples):
    for j in range(i + 1, n_samples):
        d1 = metric1_euclidean_normalized(data[i], data[j], means, stds)
        d2 = metric2_manhattan_weighted(data[i], data[j], variances)
        distances_m1.append(d1)
        distances_m2.append(d2)
        pairs.append((i, j))

distances_m1 = np.array(distances_m1)
distances_m2 = np.array(distances_m2)
pairs = np.array(pairs)

print(f"Nombre de paires calculées : {len(pairs)}")

## 4. Identification des paires les plus similaires et dissimilaires

In [ ]:
# Métrique 1 - Paires extrêmes
idx_min_m1 = np.argmin(distances_m1)
idx_max_m1 = np.argmax(distances_m1)
pair_closest_m1 = pairs[idx_min_m1]
pair_farthest_m1 = pairs[idx_max_m1]

# Métrique 2 - Paires extrêmes
idx_min_m2 = np.argmin(distances_m2)
idx_max_m2 = np.argmax(distances_m2)
pair_closest_m2 = pairs[idx_min_m2]
pair_farthest_m2 = pairs[idx_max_m2]

print("=" * 70)
print("RÉSULTATS - MÉTRIQUE 1 (Euclidienne Normalisée)")
print("=" * 70)
print(f"\nPaire la plus SIMILAIRE : échantillons {pair_closest_m1[0]} et {pair_closest_m1[1]}")
print(f"Distance : {distances_m1[idx_min_m1]:.4f} (sans unité)")
print(f"Échantillon {pair_closest_m1[0]} : {data[pair_closest_m1[0]]} cm")
print(f"Échantillon {pair_closest_m1[1]} : {data[pair_closest_m1[1]]} cm")

print(f"\nPaire la plus DISSIMILAIRE : échantillons {pair_farthest_m1[0]} et {pair_farthest_m1[1]}")
print(f"Distance : {distances_m1[idx_max_m1]:.4f} (sans unité)")
print(f"Échantillon {pair_farthest_m1[0]} : {data[pair_farthest_m1[0]]} cm")
print(f"Échantillon {pair_farthest_m1[1]} : {data[pair_farthest_m1[1]]} cm")

In [ ]:
print("=" * 70)
print("RÉSULTATS - MÉTRIQUE 2 (Manhattan Pondérée par Variance)")
print("=" * 70)
print(f"\nPaire la plus SIMILAIRE : échantillons {pair_closest_m2[0]} et {pair_closest_m2[1]}")
print(f"Distance : {distances_m2[idx_min_m2]:.4f} cm³")
print(f"Échantillon {pair_closest_m2[0]} : {data[pair_closest_m2[0]]} cm")
print(f"Échantillon {pair_closest_m2[1]} : {data[pair_closest_m2[1]]} cm")

print(f"\nPaire la plus DISSIMILAIRE : échantillons {pair_farthest_m2[0]} et {pair_farthest_m2[1]}")
print(f"Distance : {distances_m2[idx_max_m2]:.4f} cm³")
print(f"Échantillon {pair_farthest_m2[0]} : {data[pair_farthest_m2[0]]} cm")
print(f"Échantillon {pair_farthest_m2[1]} : {data[pair_farthest_m2[1]]} cm")

In [ ]:
print("=" * 70)
print("VÉRIFICATION : Les paires sont-elles différentes ?")
print("=" * 70)

closest_different = not np.array_equal(pair_closest_m1, pair_closest_m2)
farthest_different = not np.array_equal(pair_farthest_m1, pair_farthest_m2)

print(f"\nPaires les plus similaires différentes : {'✓ OUI' if closest_different else '✗ NON'}")
print(f"  - Métrique 1 : {pair_closest_m1}")
print(f"  - Métrique 2 : {pair_closest_m2}")

print(f"\nPaires les plus dissimilaires différentes : {'✓ OUI' if farthest_different else '✗ NON'}")
print(f"  - Métrique 1 : {pair_farthest_m1}")
print(f"  - Métrique 2 : {pair_farthest_m2}")

## 5. Discussion et analyse du balance des features

In [ ]:
# Analyse de la contribution de chaque feature
print("=" * 70)
print("ANALYSE DE L'ÉQUILIBRE DES FEATURES")
print("=" * 70)

feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

print("\n--- Métrique 1 (Euclidienne Normalisée) ---")
print("Après normalisation z-score, chaque feature contribue équitablement.")
print("Poids relatif de chaque feature : 25% chacune")

print("\n--- Métrique 2 (Manhattan Pondérée par Variance) ---")
total_var = np.sum(variances)
weights_m2 = variances / total_var * 100
print("Poids relatif de chaque feature (basé sur la variance) :")
for name, var, weight in zip(feature_names, variances, weights_m2):
    print(f"  - {name}: variance = {var:.3f} cm², poids = {weight:.1f}%")

In [ ]:
# Visualisation des poids
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Métrique 1 - Poids égaux
axes[0].bar(feature_names, [25, 25, 25, 25], color='steelblue')
axes[0].set_ylabel('Poids (%)')
axes[0].set_title('Métrique 1 : Euclidienne Normalisée\n(Poids égaux après z-score)')
axes[0].set_ylim(0, 60)

# Métrique 2 - Poids par variance
colors = ['green' if w > 25 else 'orange' for w in weights_m2]
axes[1].bar(feature_names, weights_m2, color=colors)
axes[1].set_ylabel('Poids (%)')
axes[1].set_title('Métrique 2 : Manhattan Pondérée\n(Poids proportionnels à la variance)')
axes[1].set_ylim(0, 60)
axes[1].axhline(y=25, color='red', linestyle='--', label='Poids uniforme (25%)')
axes[1].legend()

plt.tight_layout()
plt.savefig('metrics_weights_1_1_2.png', dpi=150)
plt.show()

## 6. Conclusion et Discussion

### Pourquoi les résultats diffèrent :

**Métrique 1 (Euclidienne Normalisée)** :
- Donne un poids égal à chaque feature (25%)
- Après normalisation, une différence de 1 écart-type est équivalente pour toutes les features
- Les paires les plus proches/éloignées sont celles qui sont similaires/différentes **sur toutes les dimensions**

**Métrique 2 (Manhattan Pondérée par Variance)** :
- La longueur des pétales (petal_length) a le poids le plus élevé (~54%)
- Les fleurs sont considérées similaires principalement si leurs **pétales ont des longueurs proches**
- Cette métrique reflète le fait que la longueur des pétales est la feature la plus discriminante dans le dataset Iris

### Prise en compte des unités :
- **Métrique 1** : En normalisant par z-score, les unités (cm) sont "annulées", permettant une comparaison équitable
- **Métrique 2** : La pondération par la variance (cm²) multipliée par la différence absolue (cm) donne une unité de cm³, cohérente dimensionnellement

### Interprétation physique :
- La métrique 1 est appropriée quand toutes les mesures sont également importantes
- La métrique 2 est appropriée quand on veut donner plus d'importance aux caractéristiques qui varient le plus dans la population